In [ ]:
# To mount
from google.colab import drive
drive.mount('/content/drive')

# # To initialize the work environment
# %cd /content/drive/My Drive/
# !git clone https://github.com/allnightlight/SIDwithIS/casestudies -b casestudies

# # To update the work environment
# %cd /content/drive/My Drive/SIDwithIS
# !git pull

%cd /content/drive/My Drive/SIDwithIS/casestudies

In [ ]:
import os
import sys
import numpy as np
sys.path.append("../framework/")
sys.path.append("../sl/")
sys.path.append("../sid/")

from builder import Builder
from loader import Loader
from mylogger import MyLogger
from sid_agent import SidAgent
from sid_agent_factory import SidAgentFactory
from sid_build_parameter import SidBuildParameter
from sid_trainer_factory import SidTrainerFactory
from store import Store
from sid_evaluator import SidEvaluator
from builtins import isinstance
from cs01_environment_factory import Cs01EnvironmentFactory
from cs01_build_parameter_factory import Cs01BuildParameterFactory
from cs01_build_parameter import Cs01BuildParameter

In [ ]:
trainingHistoryDbPath = "cs01_training_history.sqlite"

## S300: Run learning agents

### SS310: initialize an instance of builder

In [ ]:
agentFactory = SidAgentFactory()
environmentFactory = Cs01EnvironmentFactory()
trainerFactory = SidTrainerFactory()

logger = MyLogger(console_print=True)
store = Store(trainingHistoryDbPath)

builder = Builder(agentFactory=agentFactory
                  , environmentFactory=environmentFactory
                  , trainerFactory=trainerFactory
                  , store=store
                  , logger = logger)

### SS310: define instances of Sid. Build Params

### SS330: run build

In [ ]:
target_casestudy = "cs01a-20200906"
cnt = 1
nEpoch = 2**7
N1 = 2**4 # prediction horizon

while True:
    
    agentClass = str(np.random.choice(("agent001", "agent002")))
    N0 = int(np.random.choice((1, 2**2, 2**4)))
    N1 = N1
    NhiddenAgent = int(np.random.choice((2**1, 2**2, 2**4)))
    sampling_balance = float(np.random.choice((0.1, 0.5, 0.9)))
    use_offset_compensate = int(np.random.choice((0, 1)))
    use_imbalanced_sampling = int(np.random.choice((0, 1)))
    seed = int(np.random.randint(5))
    
    buildParameter = Cs01BuildParameter(
        nIntervalSave= nEpoch//2
        , nEpoch= nEpoch
        , label= target_casestudy + " %3d" % cnt
        , sampling_balance=sampling_balance
        , NhiddenAgent=NhiddenAgent
        , agentClass=agentClass
        , Nbatch=2 ** 6
        , N0=N0
        , N1=N1
        , Ntrain=2 ** 11 - 2**8
        , dampingConstantInitial=0.99
        , use_offset_compensate=use_offset_compensate
        , use_imbalanced_sampling=use_imbalanced_sampling
        , seed = seed)
    
    builder.build(buildParameter)
    cnt += 1